# Projeto 2 de Ciência dos Dados

Integrantes: Ana Carolina Pela Tofetti, Eduardo Lima Pinelli e Rafael Seicali Malcervelli

# Introdução

O universo das apostas sempre envolveu o mundo dos esportes, seja em corridas de cavalo, jogos de futebol ou lutas de boxe. No entanto, os "e-sports" ou jogos eletrônicos - videogames - vem ganhando força em diversas competições grandiosas ja realizadas; os jogos que apresentam um cenário competitivo e profissional são muitos, como: League of Legends, Starcraft, Mortal Kombat, Rainbow Six Siege, Counter Strike Global Offensive. 

Em nosso trabalho, optamos pelo jogo Counter Strike Global Offensive como ferramenta de estudo. Queremos facilitar a escolha entre dois times que irão se enfrentar em algum campeonato futuro para que assim possamos ter mais certeza em qual deles apostar.

# Explicação

Counter Strike Global Offensive é um jogo online que consiste em duas equipes se enfrentando em duelos armados entre 30 rounds. Cada equipe tem 5 integrantes e cada um joga de um lado (contra-terrorista e terrorista), sendo que se uma equipe começa o jogo como terrorista, após 15 rounds, trocam-se os lados e a equipe passa a ser contra-terrorista.

O objetivo do terrorista é plantar uma bomba em qualquer um dos bombsites disponiveis (em todos os diferentes mapas, há um bombsite A e um bombsite B). Logo, o objetivo do contra-terrorista é defender os bombsites, evitando que a equipe adversária plante a bomba e, se ela for plantada, conseguir desarmá-la antes que exploda.

Normalmente que antes mesmo de se chegar nos bombsites e ter a oportunidade de plantar a bomba, os terroristas tem de lidar com possiveis contra-terroristas em confrontos armados.

A equipe que chegar a 16 de pontuação ganha o jogo.

# Importando bibliotecas

In [34]:
#Carregando as bibliotecas
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import seaborn as sns
import statsmodels.api as sm
from collections import Counter
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\rafae\Desktop\cdados_p2\cdados_projeto2


# Leitura do dataset e limpeza do mesmo

In [30]:
#Lendo o dataset
#Criando um novo dataframe somente com as partidas, mapas e times, excluindo as estatisticas de cada jogador, pois
#existem alguns jogadores que atualmente não estão mais participando do cenário competitivo do jogo.
data = pd.read_csv("matches_info.csv").iloc[:, 0:23]

#Apagando colunas desnecessárias
del data["Unnamed: 0"]
del data["date_match"] #data da partida
del data["event_name"] #nome do evento

#Esta função faz com que todas as colunas sejam expostas pelo DataFrame.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

In [4]:
data.head(3)

,team1,team2,team_lost_score,team_won_score,maps_info,map1_played,team_winner_map1,result_map1_played1,result_half_score_map1,team_loser_map1,result_map1_played2,map2_played,team_winner_map2,result_map2_played1,result_half_score_map2,team_loser_map2,result_map2_played2,map3_played,team_winner_map3,result_map3_played1
0,Astralis,Virtus.pro,1,2,Best of 3\n\n* Grand final,Nuke,Virtus.pro,12,(6:9; 6:7),Astralis,16,Overpass,Astralis,16,(9:6; 7:8),Virtus.pro,14,Train,Astralis,16
1,Virtus.pro,SK,0,2,"Best of 3\n\n* Semi-final\n\n* Starting time is approximate, the match will start after the previous one is finished",Train,Virtus.pro,19,(10:5; 5:10) (4:2),SK,17,Cobblestone,Virtus.pro,16,(6:9; 10:5),SK,14,Overpass,NotPlayed,NotPlayed
2,Astralis,fnatic,0,2,Best of 3\n\n* Semi-final,Cache,Astralis,19,(9:6; 6:9) (4:1),fnatic,16,Nuke,Astralis,16,(12:3; 4:2),fnatic,5,Dust2,NotPlayed,NotPlayed


# Funções utilizadas ao longo do código

In [5]:
#Função que conta a quantidade de aparições dos mapas, retornando um dicionário com cada um deles e suas frequencias
def contagem(lista):
    dicionario_contagem = {}
    for i in lista:
        dicionario_contagem[i] = lista.count(i)
        
    return dicionario_contagem

#Função que recebe uma lista e retorna o item mais frequente dela.
def mais_frequente(lista): 
    contagem = Counter(lista) 
    
    return contagem.most_common(1)[0][0] 

# Selecionando algumas variáveis relevantes

In [6]:
#Criando uma lista que contenha o nome de todos os times
lista_times1 = list(data.team1)
lista_times2 = list(data.team2)
lista_times = []

for time in lista_times1:
    if time in lista_times:
        pass
    else:
        lista_times.append(time)

for time in lista_times2:
    if time in lista_times:
        pass
    else:
        lista_times.append(time)
print("Lista dos times: {0}".format(lista_times))

#Criando uma lista que contenha o nome de todos os mapas
lista_mapas = []
lista_frq_mapas = []

lista_crua = list(data.map1_played) + list(data.map2_played) + list(data.map3_played)

for i in lista_crua:
    if i in lista_mapas or i == "NotPlayed":
        pass
    else:
        lista_mapas.append(i)

print("\n\nLista dos mapas: {0}".format(lista_mapas))

#Criando um dicionário que mostra a frequencia que cada mapa é jogado, ou seja, dentre todas as partidas, 
#qual a porcentagem que cada mapa foi jogado.
for i in lista_crua:
    if i == "NotPlayed":
        pass
    else:
        lista_frq_mapas.append(i)


soma_mapas = sum(contagem(lista_frq_mapas).values())
dicionario_frequencias_mapas = {}
for k, v in contagem(lista_frq_mapas).items():
    frq = v*(100) / soma_mapas
    dicionario_frequencias_mapas[k] = frq
    
print("\n\nDicionário recorrência dos mapas [%]: {0}".format(dicionario_frequencias_mapas))

Lista dos times: ['Astralis', 'Virtus.pro', 'FaZe', 'fnatic', 'Natus Vincere', 'Liquid', 'North', 'Gambit', 'SK', 'GODSENT', 'mousesports', 'Envy', 'FlipSid3', 'OpTic', 'Immortals', 'BIG', 'G2', 'PENTA', 'Vega Squadron', 'Cloud9', 'MIBR', 'Quantum Bellator Fire', 'Space Soldiers', 'Complexity', 'NiP', 'TYLOO', 'Winstrike', 'ENCE', 'Renegades', 'NRG', 'Vitality', 'AVANGAR', 'HellRaisers', 'CR4ZY', 'DreamEaters']


Lista dos mapas: ['Nuke', 'Train', 'Cache', 'Mirage', 'Overpass', 'Dust2', 'Cobblestone', 'Inferno', 'Vertigo']


Dicionário recorrência dos mapas [%]: {'Nuke': 8.638743455497382, 'Train': 13.612565445026178, 'Cache': 7.591623036649215, 'Mirage': 20.418848167539267, 'Overpass': 14.136125654450261, 'Dust2': 11.780104712041885, 'Cobblestone': 4.973821989528796, 'Inferno': 17.277486910994764, 'Vertigo': 1.5706806282722514}


In [7]:
#Criando um dataset mais limpo, para facilitar na visualização com apenas as variáveis que nos importam.
estatisticas = data.drop(columns=["maps_info", "team_lost_score",
                                 "team_won_score", "result_map1_played1", 
                                "result_half_score_map1", "result_map1_played2",
                                "result_map2_played1", 
                                "result_half_score_map2", "result_map2_played2",
                                "result_map3_played1", 
                                "result_half_score_map1"])
estatisticas.head(3)

,team1,team2,map1_played,team_winner_map1,team_loser_map1,map2_played,team_winner_map2,team_loser_map2,map3_played,team_winner_map3
0,Astralis,Virtus.pro,Nuke,Virtus.pro,Astralis,Overpass,Astralis,Virtus.pro,Train,Astralis
1,Virtus.pro,SK,Train,Virtus.pro,SK,Cobblestone,Virtus.pro,SK,Overpass,NotPlayed
2,Astralis,fnatic,Cache,Astralis,fnatic,Nuke,Astralis,fnatic,Dust2,NotPlayed


In [8]:
#Criando um dicionário que mostra a porcentagem de vitória dos times a partir da soma de todas 
#as partidas computadas.
lista_vencedores_mapa2 = []
lista_vencedores_mapa3 = []

for i in data.team_winner_map2:
    if i == "NotPlayed":
        pass
    else:
        lista_vencedores_mapa2.append(i)

for i in data.team_winner_map3:
    if i == "NotPlayed":
        pass
    else:
        lista_vencedores_mapa3.append(i)

vencedores = list(data.team_winner_map1) + lista_vencedores_mapa2 + lista_vencedores_mapa3
soma_vencedores = sum(contagem(vencedores).values()) #A função contagem nos retorna um dicionário com as chaves
#sendo os times, e os valores a quantidade de jogos que eles ganharam.

dicionario_frequencias_vencedores = {}
dicionario_vencedores_sorted = {}
dicionario_times_sorted = {}
dicionario_vencedores_times = {}
dicionario_frequencias_times = {}

for k, v in contagem(vencedores).items():
    frq = v*(100) / soma_vencedores
    dicionario_frequencias_vencedores[k] = frq

print("\n\nDicionário frequência vencedores [%]: {0}".format(dicionario_frequencias_vencedores))

#Transformando todos as keys em lowercase, para poder ordená-las melhor.
dict_lower_vencedores = {}
for k, v in dicionario_frequencias_vencedores.items():
    key = k.lower()
    dict_lower_vencedores[key] = dicionario_frequencias_vencedores[k]
    
dict_lower_times = {}
for k, v in dicionario_frequencias_times.items():
    key = k.lower()
    dict_lower_times[key] = dicionario_frequencias_times[k]
    
dict_lower_mapas = {}
for k, v in dicionario_frequencias_mapas.items():
    key = k.lower()
    dict_lower_mapas[key] = dicionario_frequencias_mapas[k]   
    
#Organizando os dicionários em ordem alfabetica, facilitando sua visualização
dict_lower_vencedores = dict((k, v) for k, v in dict_lower_vencedores.items()) 
for key in sorted(dict_lower_vencedores):
    dicionario_vencedores_sorted[key] = dict_lower_vencedores[key]
    
dict_lower_times = dict((k, v) for k, v in dict_lower_times.items()) 
for key in sorted(dict_lower_times):
    dicionario_times_sorted[key] = dict_lower_times[key]

#Criando uma lista de listas: cada lista representa uma linha do dataframe, ou seja, um confronto.
lista_vencedores = []
lista_adicional = []
for i in range(0, 239):
    lista_adicional.append(estatisticas.iloc[ i , 3])
    lista_adicional.append(estatisticas.iloc[ i , 6])
    lista_adicional.append(estatisticas.iloc[ i , 9])
    lista_vencedores.append(lista_adicional)
    lista_adicional = []

lista_confrontos = []
lista_add = []
for i in range(0, 240):
    lista_add.append(lista_times1[i])
    lista_add.append(lista_times2[i])
    lista_confrontos.append(lista_add)
    lista_add = []

print("\n\nLista dos confrontos: {0}".format(lista_confrontos))



Dicionário frequência vencedores [%]: {'Virtus.pro': 3.857566765578635, 'Astralis': 13.353115727002967, 'FaZe': 6.824925816023739, 'fnatic': 4.451038575667655, 'North': 2.6706231454005933, 'Gambit': 4.451038575667655, 'SK': 4.154302670623146, 'GODSENT': 0.5934718100890207, 'Liquid': 5.044510385756676, 'OpTic': 0.29673590504451036, 'Natus Vincere': 8.6053412462908, 'Envy': 0.5934718100890207, 'mousesports': 2.077151335311573, 'G2': 3.857566765578635, 'Immortals': 2.373887240356083, 'BIG': 2.6706231454005933, 'FlipSid3': 0.5934718100890207, 'Cloud9': 4.451038575667655, 'PENTA': 0.29673590504451036, 'Vega Squadron': 0.8902077151335311, 'Quantum Bellator Fire': 0.8902077151335311, 'Space Soldiers': 0.5934718100890207, 'MIBR': 3.857566765578635, 'ENCE': 4.154302670623146, 'NiP': 2.077151335311573, 'Complexity': 1.7804154302670623, 'HellRaisers': 1.7804154302670623, 'TYLOO': 0.29673590504451036, 'Renegades': 3.857566765578635, 'Vitality': 2.6706231454005933, 'AVANGAR': 2.9673590504451037, 

In [9]:
#Criando uma nova coluna "team_winner", que apresenta apenas os vencedores dos confrontos.
estatisticas["team_winner"] = "Teste"
contador = 0

#Criando uma lista que contenha apenas os times que ganharam os confrontos. Como cada confronto varia 
#seu tipo por campeonato (pode ser um jogo só ou uma melhor de 3)
lista_win = []
for lista in lista_vencedores:
    lista_win.append(mais_frequente(lista))

for mapa in estatisticas["map2_played"]:
    #Se o mapa 2 representar um "NotPlayed", significa que o confronto é de um jogo apenas, 
    #então, o vencedor do primeiro jogo é o vencedor do confronto
    if mapa == "NotPlayed":
        estatisticas["team_winner"][contador] = estatisticas["team_winner_map1"][contador]
        contador += 1
        
    else:
        estatisticas["team_winner"][contador] = lista_win[contador]
        contador += 1

In [10]:
estatisticas.head(5)

,team1,team2,map1_played,team_winner_map1,team_loser_map1,map2_played,team_winner_map2,team_loser_map2,map3_played,team_winner_map3,team_winner
0,Astralis,Virtus.pro,Nuke,Virtus.pro,Astralis,Overpass,Astralis,Virtus.pro,Train,Astralis,Astralis
1,Virtus.pro,SK,Train,Virtus.pro,SK,Cobblestone,Virtus.pro,SK,Overpass,NotPlayed,Virtus.pro
2,Astralis,fnatic,Cache,Astralis,fnatic,Nuke,Astralis,fnatic,Dust2,NotPlayed,Astralis
3,FaZe,SK,Mirage,FaZe,SK,Train,SK,FaZe,Overpass,SK,SK
4,Virtus.pro,North,Overpass,Virtus.pro,North,Cache,North,Virtus.pro,Cobblestone,Virtus.pro,Virtus.pro


In [11]:
#Criando um dicionário que contenha como chave os times e como valor, outro dicionário que contenha todos os mapas
#disponiveis para jogar (lista_mapas) e na sua chave, as porcentagens que cada time ganhou nesse mapa, dado todos os
#jogos que ja jogou nele antes.

#Criando listas suporte
lista_time1 = []
lista_time2 = []
lista_ganhou = []
lista_mapas_confrontos = []

#Alimentando as listas suporte
for i in range(240):
    
    confronto2 = estatisticas.iloc[i, 6] != "NotPlayed"
    confronto3 = estatisticas.iloc[i, 9] != "NotPlayed"
    
    lista_time1.append(estatisticas.iloc[i, 0])
    lista_time2.append(estatisticas.iloc[i, 1])
    lista_ganhou.append(estatisticas.iloc[i,3])
    lista_mapas_confrontos.append(estatisticas.iloc[i,2])
    
    if confronto2:
        lista_time1.append(estatisticas.iloc[i, 0])
        lista_time2.append(estatisticas.iloc[i, 1])
        lista_ganhou.append(estatisticas.iloc[i,6])
        lista_mapas_confrontos.append(estatisticas.iloc[i,5])
        
    if confronto3:
        lista_time1.append(estatisticas.iloc[i, 0])
        lista_time2.append(estatisticas.iloc[i, 1])
        lista_ganhou.append(estatisticas.iloc[i,9])
        lista_mapas_confrontos.append(estatisticas.iloc[i,8])

#Criando a estrutura do dicionário principal e outros 2 dicionários suporte
dicionario_vitorias_mapas = {}

for time in lista_times:
    dicionario_vitorias_mapas[time] = {}
    for mapa in lista_mapas:
        dicionario_vitorias_mapas[time][mapa] = 0
        
dicionario_vezes_ganhou = {}
for time in lista_times:
    dicionario_vezes_ganhou[time] = {}
    for mapa in lista_mapas:
        dicionario_vezes_ganhou[time][mapa] = 0
        
dicionario_vezes_jogou = {}
for time in lista_times:
    dicionario_vezes_jogou[time] = {}
    for mapa in lista_mapas:
        dicionario_vezes_jogou[time][mapa] = 0

#Alimentando os dicionários suporte com os dados
for i in range(len(lista_mapas_confrontos)):
    
    time1 = lista_time1[i]
    time2 = lista_time2[i]
    ganhador = lista_ganhou[i]
    mapa = lista_mapas_confrontos[i]
    
    dicionario_vezes_jogou[time1][mapa] += 1
    dicionario_vezes_jogou[time2][mapa] += 1
    
    if time1 == ganhador:
        dicionario_vezes_ganhou[time1][mapa] += 1
    else:
        dicionario_vezes_ganhou[time2][mapa] += 1

#Usando os dois dicionários suportes para alimentar o dicionário principal.
for time in dicionario_vezes_jogou:
    dicmapas =  dicionario_vezes_jogou[time]
    for nomemapa in dicmapas:
        frqcmapa = dicmapas[nomemapa]
        if frqcmapa == 0:
            dicionario_vitorias_mapas[time][nomemapa] = 0
        else:
            dicionario_vitorias_mapas[time][nomemapa] = dicionario_vezes_ganhou[time][nomemapa]*100/dicionario_vezes_jogou[time][nomemapa]
            
print("Dicionário porcentagem de vitória em cada mapa [%]: {0}".format(dicionario_vitorias_mapas))

Dicionário porcentagem de vitória em cada mapa [%]: {'Astralis': {'Nuke': 77.77777777777777, 'Train': 63.63636363636363, 'Cache': 66.66666666666667, 'Mirage': 57.142857142857146, 'Overpass': 88.88888888888889, 'Dust2': 71.42857142857143, 'Cobblestone': 0, 'Inferno': 83.33333333333333, 'Vertigo': 100.0}, 'Virtus.pro': {'Nuke': 100.0, 'Train': 60.0, 'Cache': 33.333333333333336, 'Mirage': 0.0, 'Overpass': 50.0, 'Dust2': 0, 'Cobblestone': 100.0, 'Inferno': 0.0, 'Vertigo': 0}, 'FaZe': {'Nuke': 100.0, 'Train': 50.0, 'Cache': 80.0, 'Mirage': 53.333333333333336, 'Overpass': 0.0, 'Dust2': 60.0, 'Cobblestone': 0, 'Inferno': 16.666666666666668, 'Vertigo': 0.0}, 'fnatic': {'Nuke': 0.0, 'Train': 0.0, 'Cache': 16.666666666666668, 'Mirage': 71.42857142857143, 'Overpass': 33.333333333333336, 'Dust2': 66.66666666666667, 'Cobblestone': 100.0, 'Inferno': 57.142857142857146, 'Vertigo': 0}, 'Natus Vincere': {'Nuke': 40.0, 'Train': 75.0, 'Cache': 0, 'Mirage': 50.0, 'Overpass': 44.44444444444444, 'Dust2': 62

In [12]:
#Criando um dicionário que contenha como chave os times e como valor, outro dicionário que contenha todos os outros
#times disponiveis para jogar (lista_times) e na sua chave, as porcentagens de vitória contra cada time, dado todos os
#jogos que ja jogou contra ele antes.

dicionario_vitorias_times = {}
for time in lista_times:
    dicionario_vitorias_times[time] = {}
    for timeadv in lista_times:
        if timeadv == time:
            continue
        else:
            dicionario_vitorias_times[time][timeadv] = 0
            
dicionario_vezes_jogou_time = {}
for time in lista_times:
    dicionario_vezes_jogou_time[time] = {}
    for timeadv in lista_times:
        if timeadv == time:
            continue
        else:
            dicionario_vezes_jogou_time[time][timeadv] = 0
        
dicionario_vezes_ganhou_time = {}
for time in lista_times:
    dicionario_vezes_ganhou_time[time] = {}
    for timeadv in lista_times:
        if timeadv == time:
            continue
        else:
            dicionario_vezes_ganhou_time[time][timeadv] = 0
            
for i in range(len(lista_ganhou)):
    
    time1 = lista_time1[i]
    time2 = lista_time2[i]
    ganhador = lista_ganhou[i]
    
    dicionario_vezes_jogou_time[time1][time2] += 1
    dicionario_vezes_jogou_time[time2][time1] += 1
    
    if time1 == ganhador:
        dicionario_vezes_ganhou_time[time1][time2] += 1
    else:
        dicionario_vezes_ganhou_time[time2][time1] += 1

for time in dicionario_vezes_jogou_time:
    dicadv =  dicionario_vezes_jogou_time[time]
    for adv in dicadv:
        frqcadv = dicadv[adv]
        if frqcadv == 0:
            dicionario_vitorias_times[time][adv] = 0
        else:
            dicionario_vitorias_times[time][adv] = dicionario_vezes_ganhou_time[time][adv]*100/dicionario_vezes_jogou_time[time][adv]
            
print("Dicionário porcentagem de vitória em relação a cada time [%]: {0}".format(dicionario_vitorias_times))

Dicionário porcentagem de vitória em relação a cada time [%]: {'Astralis': {'Virtus.pro': 66.66666666666667, 'FaZe': 100.0, 'fnatic': 75.0, 'Natus Vincere': 83.33333333333333, 'Liquid': 83.33333333333333, 'North': 100.0, 'Gambit': 33.333333333333336, 'SK': 50.0, 'GODSENT': 0.0, 'mousesports': 0.0, 'Envy': 0, 'FlipSid3': 0, 'OpTic': 100.0, 'Immortals': 100.0, 'BIG': 0, 'G2': 100.0, 'PENTA': 0, 'Vega Squadron': 100.0, 'Cloud9': 50.0, 'MIBR': 100.0, 'Quantum Bellator Fire': 0, 'Space Soldiers': 0, 'Complexity': 100.0, 'NiP': 100.0, 'TYLOO': 0, 'Winstrike': 0, 'ENCE': 100.0, 'Renegades': 66.66666666666667, 'NRG': 50.0, 'Vitality': 0, 'AVANGAR': 100.0, 'HellRaisers': 0, 'CR4ZY': 66.66666666666667, 'DreamEaters': 100.0}, 'Virtus.pro': {'Astralis': 33.333333333333336, 'FaZe': 0, 'fnatic': 50.0, 'Natus Vincere': 0, 'Liquid': 0, 'North': 66.66666666666667, 'Gambit': 50.0, 'SK': 100.0, 'GODSENT': 0, 'mousesports': 0, 'Envy': 0, 'FlipSid3': 0, 'OpTic': 100.0, 'Immortals': 0.0, 'BIG': 0, 'G2': 100

# Transformando algumas variáveis

In [13]:
#Transformando os dicionários que acima foram criados em dataframes.
df_mapas = pd.DataFrame.from_dict(dicionario_vitorias_mapas)
df_times = pd.DataFrame.from_dict(dicionario_vitorias_times)

In [14]:
df_mapas.head(3)

,Astralis,Virtus.pro,FaZe,fnatic,Natus Vincere,Liquid,North,Gambit,SK,GODSENT,mousesports,Envy,FlipSid3,OpTic,Immortals,BIG,G2,PENTA,Vega Squadron,Cloud9,MIBR,Quantum Bellator Fire,Space Soldiers,Complexity,NiP,TYLOO,Winstrike,ENCE,Renegades,NRG,Vitality,AVANGAR,HellRaisers,CR4ZY,DreamEaters
Cache,66.666667,33.333333,80.0,16.666667,0.0,50.0,100.000000,50.0,0.0,50.0,66.666667,100.0,0.0,0.0,0.0,0.000000,60.0,0.0,0.0,33.333333,100.0,50.0,0.0,100.0,50.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,25.0,0.0,0.0
Cobblestone,0.000000,100.000000,0.0,100.000000,100.0,0.0,16.666667,50.0,50.0,0.0,0.000000,0.0,0.0,0.0,50.0,66.666667,100.0,0.0,0.0,0.000000,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dust2,71.428571,0.000000,60.0,66.666667,62.5,60.0,50.000000,0.0,50.0,0.0,0.000000,0.0,0.0,0.0,0.0,66.666667,60.0,0.0,0.0,0.000000,60.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,20.0,100.0,25.0,40.0,75.0,100.0,0.0


In [15]:
df_times.head(3)

,Astralis,Virtus.pro,FaZe,fnatic,Natus Vincere,Liquid,North,Gambit,SK,GODSENT,mousesports,Envy,FlipSid3,OpTic,Immortals,BIG,G2,PENTA,Vega Squadron,Cloud9,MIBR,Quantum Bellator Fire,Space Soldiers,Complexity,NiP,TYLOO,Winstrike,ENCE,Renegades,NRG,Vitality,AVANGAR,HellRaisers,CR4ZY,DreamEaters
AVANGAR,100.0,0.000000,0.0,0.0,100.000000,50.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,33.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.000000,0.000000,0.0,33.333333,NaN,0.0,0.000000,0.0
Astralis,NaN,33.333333,0.0,25.0,16.666667,16.666667,0.0,66.666667,50.0,100.0,100.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,33.333333,50.0,0.000000,0.0,0.0,33.333333,0.0
BIG,0.0,0.000000,0.0,0.0,100.000000,100.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,66.666667,NaN,100.000000,0.0,0.0,0.0,0.0,0.0,100.0,100.0,0.0,0.0,0.0,66.666667,0.000000,0.0,100.000000,0.0,0.0,0.000000,0.0


In [16]:
#Criando um novo dataframe, substituindo as strings por numeros, para possibilitar o uso do classifica
if "NotPlayed" in lista_times:
    lista_times.pop(0)
else:
    lista_times.insert(0, "NotPlayed")
mapteam = dict(enumerate(lista_times))
reversed_mapteam = {value : key for (key, value) in mapteam.items()}
estat_time_n = estatisticas.applymap(lambda s: reversed_mapteam.get(s) if s in reversed_mapteam else s)

if "NotPlayed" in lista_mapas:
    lista_mapas.pop(0)
else:
    lista_mapas.insert(0, "NotPlayed")
mapmapas = dict(enumerate(lista_mapas))
reversed_mapmapas = {value : key for (key, value) in mapmapas.items()}
estat_time_map_n = estat_time_n.applymap(lambda s: reversed_mapmapas.get(s) if s in reversed_mapmapas else s)

# Legenda dos times e mapas no novo dataframe:

0: NotPlayed //
1: Astralis -- Nuke//
2: Virtus.pro -- Train //
3: Faze -- Cache //
4: fnatic -- Mirage //
5: Natus Vincere -- Overpass //
6: Liquid -- Dust2//
7: North -- Cobblestone //
8: Gambit -- Inferno //
9: SK -- Vertigo //
10: GODSENT //
11: mousesports //
12: Envy //
13: FlipSid3 //
14: OpTic //
15: Immortals //
16: BIG //
17: G2 //
18: PENTA //
19: Vega Squadron //
20: Cloud9 //
21: MIBR //
22: Quantum Bellator Fire //
23: Space Soldiers //
24: Complexity //
25: NiP //
26: TYLOO //
27: Winstrike //
28: ENCE //
29: Renegades //
30: NRG //
31: Vitality //
32: AVANGAR //
33: Hellraisers //
34: CR4ZY //
35: DreamEaters //

In [18]:
estat_time_map_n.head(7)

,team1,team2,map1_played,team_winner_map1,team_loser_map1,map2_played,team_winner_map2,team_loser_map2,map3_played,team_winner_map3,team_winner
0,1,2,1,2,1,5,1,2,2,1,1
1,2,9,2,2,9,7,2,9,5,0,2
2,1,4,3,1,4,1,1,4,6,0,1
3,3,9,4,3,9,2,9,3,5,9,9
4,2,7,5,2,7,3,7,2,7,2,2
5,4,8,3,4,8,5,8,4,6,4,4
6,5,1,5,1,5,4,5,1,6,1,1


# Criando o classificador

In [42]:
#Criando uma nova coluna no dataframe que representa um valor True or False(em formato binomial, 0 e 1).
estat_time_map_n["team1_wins"] = (estat_time_map_n["team1"] == estat_time_map_n["team_winner"]).astype(int)

In [44]:
#Selecionando as colunas para teste e treino, além de decidir as variáveis importantes.
X = estat_time_map_n.drop(["team_winner", "team1_wins"], axis=1)
Y = estat_time_map_n["team1_wins"]
X.head(3)

,team1,team2,map1_played,team_winner_map1,team_loser_map1,map2_played,team_winner_map2,team_loser_map2,map3_played,team_winner_map3
0,1,2,1,2,1,5,1,2,2,1
1,2,9,2,2,9,7,2,9,5,0
2,1,4,3,1,4,1,1,4,6,0


In [45]:
#Utilizando o método train_test_split, para dividir nosso dataframe em dados de treino e de teste, visto que não deve
#utilizar todos os dados para o treino.
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

# Treinando o classificador

In [46]:
#Criando o classificador
classifica = RandomForestClassifier(n_estimators = 100, random_state = 0) 
classifica.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

# Testando o classificador

In [47]:
#Calculando a acurácia do nosso classificador.
y_pred = classifica.predict(X_test)
print("Acurácia do classificador: {0}".format(accuracy_score(y_test, y_pred)))

Acurácia do classificador: 0.7916666666666666


# Conclusão

A partir dos resultados obtidos, conseguimos criar um classificador a partir do método de RandomForest, com uma acurácia de ~80% de acerto. 

Para possíveis melhorias no futuro, o classificador poderia ser alimentado com estatísticas individuais de cada jogador, como granadas lançadas, porcentagem de dano realizado por rodada, abates e mortes etc.